In [1]:
!pip install rpy2==3.5.1
%load_ext rpy2.ipython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached rpy2-3.5.1-cp37-cp37m-linux_x86_64.whl
  Attempting uninstall: rpy2
    Found existing installation: rpy2 3.5.5
    Uninstalling rpy2-3.5.5:
      Successfully uninstalled rpy2-3.5.5


<div dir='rtl'>
<h1>1.احتمال شرطی (Conditional Probability)</h1>
</div>

<div dir='rtl'>
فرض کنید که یک آزمایش کرونا در ۹۰درصد مواقع کسی که واقعا مبتلا به کرونا هست را مثبت پاسخ میدهد و در ۹۵درصد مواقع کسی که مبتلا نیست را پاسخ منفی میدهد.

به عبارتی داریم:
</div>

<div dir='ltr'>
P(test pos | Corona) = 0.9<br>
P(test neg | no Corona) = 0.95
</div>

<div dir='rtl'>
همچنین میدانیم کرونا حدود ۱۰درصد جمعیت را درگیر کرده است.
</div>

<div dir='ltr'>
P(Corona) = 0.1
</div>

<div dir='rtl'>
شما از این آزمایش استفاده کردید و نتیجه تستتان مثبت شده.

با چه احتمالی واقعا مبتلا به کرونا هستید؟ 0.67

اگر تست منفی بود چطور؟ 0.12

مسئله را به صورت تئوری با کمک قاعده بیز حل کنید.
</div>

<div dir='rtl'>
برای اینکه دید بهتری به مسئله پیدا کنیم میتوان از زبان R و نمونه برداری کمک گرفت.

ابتدا یک جامعه نمونه ۱۰۰۰۰۰ نفری با کمک تابع sample بسازید.

دقت کنید که احتمال ابتلا به کرونا ۱۰درصد است و اعضای جامعه یا کرونا دارند و یا ندارد.
</div>

In [2]:
%%R

corona <- sample(c('No','Yes'), size=100000, replace=TRUE, prob=c(0.9,0.1))


<div dir='rtl'>
حال با کمک sample یک مجموعه نتایج تست بسازید.
به این صورت که....
</div>

In [3]:
%%R

test_result <- rep(NA, 100000)
test_result[corona=='No'] <- sample(c('Neg','Pos'), size=sum(corona=='No'), replace=TRUE, prob=c(0.95,0.05))
test_result[corona=='Yes'] <- sample(c('Neg','Pos'), size=sum(corona=='Yes'), replace=TRUE, prob=c(0.1, 0.9))

<div dir='rtl'>
حال با کمک داده هایی که با نمونه برداری بدست آوردید به سوال اول پاسخ دهید.

با وجود مثبت شدن تست کرونای شما، احتمال اینکه واقعا به کرونا مبتلا باشید چقدر است؟
</div>

In [4]:
%%R
mean(corona[test_result=='Pos']=='Yes')

[1] 0.6717307


<div dir='rtl'>
بررسی کنید که این احتمال بدون تست چقدر تغییر میکند.

آیا در بررسی نتایج تست کرونا بهتر است به احتمال نهایی که بدست آمده توجه کنیم و یا به افزایش احتمال ابتلا نسبت به حالتی که تست ندادیم؟
</div>

In [5]:
%%R
mean(corona[test_result=='Pos']=='Yes')/mean(corona=='Yes')

[1] 6.692545


<div dir='rtl'>
<h1>2. مسئله مونتی هال (Monty Hall Problem)</h1>

در درس با مسئله مونتی هال آشنا شدیم.

در این تمرین از شما انتظار میرود که به کمک نمونه برداری این مسئله را به صورت شهودی بررسی کنید.

سعی کنید مسئله را برای تعداد در های ۳،۴ و ۱۰۰ بررسی کنید و نتایج هر بخش را گزارش کنید.
</div>

In [6]:
%%R
# Inputs:
#    make_switch: TRUE or FALSE based on the given choice of the player.
#                 If TRUE, the player wants a switch.
# Output of function determines that the player is a winner or not. 
# Monty Hall game for 100 doors...

simulate_game = function(make_switch = FALSE) 
{
  n_doors = 3

  # Create a integer vector for the door indices.
  doors = seq_len(n_doors)
  
  # Create a boolean vector showing which doors are opened.
  # All doors are closed at the start of the game.
  is_closed = rep(TRUE, length=n_doors)
  
  # Sample one index for the door to hide the prize behind.
  # From now on, we call it the prize door. 
  prize_index = sample(doors, size=1)
  
  # Sample one index for the door initially chosen by the player.
  # This can be the same door as the prize door. So there is no need to 
  # eliminate any door.
  choice_index = sample(doors, size=1)
  
  # Now Monty Hall opens the door the player did not choose.
  # If the player has already picked the prize door, the one remaining 
  # closed door has a no prize.
  # If the player has not picked the prize door, the one remaining closed 
  # door has the prize.
     # If the player has the prize, Monty Hall has 99 options to choose from.
    # The door which the player has chosen (the prize door) needs to be 
    # eliminated.  
    # Else, Monty Hall has 98 remaining doors to choose from.
    # We eliminate the chosen and the prize door. 
  is_closed[sample(doors[doors != prize_index & doors != choice_index], size=1)] = FALSE
 

  # Now Monty Hall asks the player whether he/she wants to make a switch
  if (make_switch) {
    # If the player decides to make a switch, then he/she has to pick 
    # one of the closed doors.
    # We need to make sure it is not the one the player chose initially. 
    choice_index = sample(doors[is_closed & doors != choice_index], size=1)
  }
  
  # Return a boolean value showing whether the player's choice is the prize door
  # or not.
  return(choice_index == prize_index)
}

In [7]:
%%R
# Inputs:
#    make_switch: TRUE or FALSE based on the given choice of the player.
#                 If TRUE, the player wants a switch.
#    n: Number of games we want to simulate. 
# Output of function determines that the player is a winner or not in all games.

simulate_n_games = function(n, make_switch) 
{
  return(vapply(rep(make_switch, n), simulate_game, logical(1)))
}

In [8]:
%%R

n_games = 100

# Simulate games and store the boolean results in both cases.
results_without_switching = simulate_n_games(n = n_games, make_switch = FALSE)
results_with_switching = simulate_n_games(n = n_games, make_switch = TRUE)

# Store the cumulative wins in a dataframe.
results = data.frame(
  # First column
  game = seq_len(n_games), 
  # Second column
  cumulative_wins_without_switching = cumsum(results_without_switching),
  # Third column
  cumulative_wins_with_switching = cumsum(results_with_switching)
)

# Printing ten rows of stored data.
head(results, 10)

   game cumulative_wins_without_switching cumulative_wins_with_switching
1     1                                 1                              0
2     2                                 1                              0
3     3                                 1                              0
4     4                                 2                              1
5     5                                 2                              2
6     6                                 2                              2
7     7                                 2                              2
8     8                                 3                              2
9     9                                 3                              2
10   10                                 3                              2
